# Limpieza de Datos Dataset Promociones-20190715

In [3]:
import pandas as pd
from utils_functions import convert_file, upload_to_bucket

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [4]:
promo_col_names = [
        'idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'ventas_estimadas_und',
        'nro_prom', 'fecha_trans', 'porcentaje_dcto'
]

In [5]:
promo_df = pd.read_csv("../data/VMI_Promociones-20190715.txt",
                       encoding="utf-8",
                       header=None,
                       delimiter='\t',
                       names=promo_col_names)

In [6]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [7]:
promo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   idb                   285253 non-null  int64  
 1   id_producto           285253 non-null  int64  
 2   fecha_desde           285253 non-null  int64  
 3   fecha_hasta           285253 non-null  int64  
 4   ventas_estimadas_und  285253 non-null  float64
 5   nro_prom              285253 non-null  int64  
 6   fecha_trans           285253 non-null  int64  
 7   porcentaje_dcto       285253 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.4 MB


#### Ajustes al dataframe
- Convertir la columnas fecha_desde, fecha_hasta y fecha_trans a tipo de dato datetime.
- Verificar si existen valores nulos y/o repetidos.
- Obtener las fechas máximas y minimas de transaciones.
- Crear una nueva columnas llamada duracion_promo para obtener el delta de tiempo entre la fecha_desde y la fecha_hasta de una promoción.
- Obtener una análisis descriptivo de los datos.

In [8]:
promo_df = convert_file(promo_df, 'fecha_desde',  'date')

In [9]:
promo_df = convert_file(promo_df, 'fecha_hasta',  'date')

In [10]:
promo_df = convert_file(promo_df, 'fecha_trans', 'date')

In [11]:
promo_df.isnull().sum(axis=0)

idb                     0
id_producto             0
fecha_desde             0
fecha_hasta             0
ventas_estimadas_und    0
nro_prom                0
fecha_trans             0
porcentaje_dcto         0
dtype: int64

In [12]:
# Identify the duplicate rows using a subset of columns
subset_cols = ['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom', 'ventas_estimadas_und']
duplicate_mask = promo_df.duplicated(subset=subset_cols, keep='first')

# Count the number of duplicate rows
total_duplicates = sum(duplicate_mask)

# Remove the duplicate rows from promo_df
promo_df = promo_df[~duplicate_mask]

print(f"El total de registros duplicados en el dataset es: {total_duplicates}")
print(f"El total de registros después de eliminar duplicados es: {len(promo_df)}")


El total de registros duplicados en el dataset es: 268557
El total de registros después de eliminar duplicados es: 16696


Observamos que existen 268557 registros duplicados en la tabla de promociones. Esto significa que estos registros comparten la misma fecha_desde y fecha_hasta, el mismo id_producto y idb, y el mismo número de promoción y número de ventas estimadas en la promoción, diferenciándose únicamente por el valor de fecha de transacción. Por lo tanto, podemos concluir con certeza que estos registros son duplicados.

Es importante tener en cuenta que esta tabla no registra transacciones, sino que registra el número de promoción, la fecha de inicio y final, y el valor estimado de las ventas en la promoción. Por esta razón, la fecha de transacción no es relevante en este caso.

In [14]:
promo_df['duracion_promo'] = (promo_df.fecha_hasta - promo_df.fecha_desde).astype('timedelta64[D]').astype(int)

In [15]:
# group the dataframe by all columns except 'ventas_estimadas_und'
grouped = promo_df.groupby(['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom',  'duracion_promo'], as_index=False)

# sum the 'ventas_estimadas_und' column for each group
summed = grouped.agg({'ventas_estimadas_und': 'sum'})

# drop the duplicated rows
result = summed.drop_duplicates()

result.head()



,idb,id_producto,fecha_desde,fecha_hasta,nro_prom,duracion_promo,ventas_estimadas_und
0,242,228217,2016-10-23,2016-10-30,436830,7,0.98
1,242,228217,2016-11-21,2016-11-28,440561,7,60.08
2,242,240911,2016-07-23,2016-07-30,425200,7,6.51
3,242,240911,2016-09-18,2016-09-25,432776,7,25.50
4,242,240911,2016-10-23,2016-10-30,436830,7,24.11


In [17]:
result[(result['idb']==5202) & (result['id_producto']==427444) ]

,idb,id_producto,fecha_desde,fecha_hasta,nro_prom,duracion_promo,ventas_estimadas_und
2601,5202,427444,2016-08-16,2016-08-23,428908,7,61.47
2602,5202,427444,2017-01-16,2017-01-23,446144,7,53.56
2603,5202,427444,2017-04-17,2017-04-24,456807,7,62.52
2604,5202,427444,2017-09-19,2017-09-26,473864,7,44.03
2605,5202,427444,2017-11-21,2017-11-28,480036,7,39.00
2606,5202,427444,2018-03-18,2018-03-25,491206,7,34.09
2607,5202,427444,2018-04-21,2018-04-28,493122,7,38.01
2608,5202,427444,2019-03-17,2019-03-24,527744,7,5.88


In [ ]:
min_date = promo_df['fecha_desde'].min()
max_date = promo_df['fecha_hasta'].max()


print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

In [ ]:
min_date = promo_df['fecha_trans'].min()
max_date = promo_df['fecha_trans'].max()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Luego de una revisión cuidadosa de los datos, hemos encontrado que la columna 'porcentaje_dcto' parece contener información duplicada con la columna 'duracion_promo'. Aunque inicialmente obtuvimos el valor de 'porcentaje_dcto' directamente del archivo de texto, no estamos seguros de su precisión y consideramos la columna 'duracion_promo' como una mejor fuente de información para inferir el nivel de agresividad de la promoción, tal como se describe en la estructura de datos.

Dado que las dos columnas contienen información similar y que 'duracion_promo' es una mejor fuente de información, hemos decidido eliminar la columna 'porcentaje_dcto' y conservar solo 'duracion_promo' como la columna representativa de la duración de la promoción.

In [ ]:
promo_df.drop('porcentaje_dcto', axis=1, inplace=True)

In [ ]:
promo_df.describe()

#### Observaciones dataset Promociones-20190715

1. El conjunto de datos contiene información sobre promociones realizadas desde el 19 de julio de 2016 hasta el 16 de julio de 2019.

2. Hay un total de 16,696 registros y 8 columnas en el conjunto de datos, que se describen de la siguiente manera:

    idb: Identificador interno de Boca
    id_producto: Identificador interno del artículo
    fecha_desde: Fecha de inicio de la promoción
    fecha_hasta: Fecha de finalización de la promoción
    ventas_estimadas_und: Estimación de ventas en unidades durante la promoción
    nro_prom: Número de la promoción
    fecha_trans: Fecha de la información
    duracion_promo: Duración de la promoción (diferencia entre fecha_desde y fecha_hasta)

3. Observamos que hay 268,557 registros duplicados en la tabla de promociones. Estos registros comparten la misma fecha_desde,
fecha_hasta, id_producto, idb, nro_prom y ventas_estimadas_und, diferenciándose únicamente por el valor de fecha_transacción. Por lo tanto, podemos concluir con certeza que estos registros son duplicados.

4. Es importante tener en cuenta que esta tabla no registra transacciones, sino que registra el número de promoción, la fecha de inicio y final, y el valor estimado de las ventas en la promoción. Por esta razón, la fecha de transacción no es relevante en este caso.

5. La media de ventas estimadas en unidades durante las promociones es de 20.88 unidades.

6. La duración máxima de una promoción es de 18 días, mientras que la duración mínima es de 8 días, la media es de 8 días.

### Guardar el dataset limpio

In [ ]:
promo_df.to_csv('../clean_data/promociones_clean.csv', index=False)

#### Subir csv a google cloud storage

In [ ]:
upload_to_bucket('deodorants', '../clean_data/promociones_clean.csv', 'promociones')